In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

session = SparkSession(sc)
session.conf.set("spark.sql.shuffle.partitions", 5)
sample_df = session.read.format("csv")\
    .option("header", "true")\
    .option("mode", "DROPMALFORMED")\
    .option("inferSchema", "true")\
    .option("sep", "\t")\
    .load("/FileStore/tables/*.tsv")
print(sample_df.count())


4994305

In [2]:
max_help = sample_df.groupby().max('helpful_votes').first().asDict()['max(helpful_votes)']
max_sent = sample_df.groupby().max('sentiment_rating').first().asDict()['max(sentiment_rating)']
max_sub = sample_df.groupby().max('subjectivity_rating').first().asDict()['max(subjectivity_rating)']
print("Max help = %d Max sent = %d Max sub = %d" % (max_help,max_sent,max_sub))


Max help = 2591 Max sent = 1 Max sub = 1

In [3]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.sql.functions import col, countDistinct
va = VectorAssembler(inputCols = ["helpful_votes","sentiment_rating", "subjectivity_rating","word_count","sentence_count","adjective_count","adverb_count", "noun_count","pronoun_count","verb_count"], outputCol = "features")
new_df = va.transform(sample_df)
kmeans = KMeans(k=3, seed=1)
model = kmeans.fit(new_df.select('features'))
clustered = model.transform(new_df)
# clustered.show()
# clustered.agg(countDistinct(col("prediction")).alias("count")).show()

In [4]:
cl_1 = clustered.filter(clustered.prediction==0)
cl_1_stats = cl_1.select("helpful_votes", "sentiment_rating", "subjectivity_rating","word_count","sentence_count","adjective_count","adverb_count", "noun_count","pronoun_count","verb_count","prediction")
cl_1_stats.describe().show()

cl_2 = clustered.filter(clustered.prediction==1)
cl_2_stats = cl_2.select("helpful_votes", "sentiment_rating", "subjectivity_rating","word_count","sentence_count","adjective_count","adverb_count", "noun_count","pronoun_count","verb_count","prediction")
cl_2_stats.describe().show()

cl_3 = clustered.filter(clustered.prediction==2)
cl_3_stats = cl_3.select("helpful_votes", "sentiment_rating", "subjectivity_rating","word_count","sentence_count","adjective_count","adverb_count", "noun_count","pronoun_count","verb_count","prediction")
cl_3_stats.describe().show()

# cl_4 = clustered.filter(clustered.prediction==3)
# cl_4_stats = cl_4.select("helpful_votes", "sentiment_rating", "subjectivity_rating","word_count","sentence_count","adjective_count","adverb_count", "noun_count","pronoun_count","verb_count","prediction")
# cl_4_stats.describe().show()

# cl_5 = clustered.filter(clustered.prediction==4)
# cl_5_stats = cl_5.select("helpful_votes", "sentiment_rating", "subjectivity_rating","word_count","sentence_count","adjective_count","adverb_count", "noun_count","pronoun_count","verb_count","prediction")
# cl_5_stats.describe().show()

+-------+------------------+-------------------+-------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+----------+
summary| helpful_votes| sentiment_rating|subjectivity_rating| word_count| sentence_count| adjective_count| adverb_count| noun_count| pronoun_count| verb_count|prediction|
+-------+------------------+-------------------+-------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+----------+
 count| 557360| 557360| 557360| 557360| 557360| 557360| 557360| 557360| 557360| 557360| 557360|
 mean| 3.9285129898091|0.17003801776970248| 0.5196656395319502|170.01243720396153| 9.19955325104062|15.253032151571695|13.269585187311613|16.33366944165351|15.253032151571695| 33.68487153724702| 0.0|
 stddev|19.001090575972317|0.12026780698007226|0.09653671847158582| 68.76259815258526|4.3003588574687885| 7.697485852123997| 6.632759612305376|7.965517673778051| 7.697485852123997|14.045527055253357| 0.0|
 min| 0| -1.0| 0.0| 96| 1| 0| 0| 0| 0| 0| 0|
 max| 4004| 1.0| 1.0| 436| 54| 111| 56| 73| 111| 110| 0|
+-------+------------------+-------------------+-------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+----------+

+-------+------------------+-----------------+-------------------+------------------+------------------+-----------------+------------------+-----------------+-----------------+------------------+----------+
summary| helpful_votes| sentiment_rating|subjectivity_rating| word_count| sentence_count| adjective_count| adverb_count| noun_count| pronoun_count| verb_count|prediction|
+-------+------------------+-----------------+-------------------+------------------+------------------+-----------------+------------------+-----------------+-----------------+------------------+----------+
 count| 4407405| 4407405| 4407405| 4407405| 4407405| 4407405| 4407405| 4407405| 4407405| 4407405| 4407405|
 mean|0.7583671117131282|0.327664589387788| 0.5821592625517626|30.893335420729432| 2.714856020719675|3.147207483768794|2.4821406246986606|3.316044928932104|3.147207483768794| 6.075369973941583| 1.0|
 stddev| 3.807869429995705|0.305992888443447|0.23689468307169875| 24.02313118683606|1.7340991349170383|2.728713028565038|2.4948783849044553| 3.24358600073478|2.728713028565038|5.2736300938732095| 0.0|
 min| 0| -1.0| 0.0| 0| 0| 0| 0| 0| 0| 0| 1|
 max| 1372| 1.0| 1.0| 106| 44| 28| 42| 28| 28| 33| 1|
+-------+------------------+-----------------+-------------------+------------------+------------------+-----------------+------------------+-----------------+-----------------+------------------+----------+

+-------+------------------+--------------------+--------------------+-----------------+------------------+------------------+-----------------+------------------+------------------+-----------------+----------+
summary| helpful_votes| sentiment_rating| subjectivity_rating| word_count| sentence_count| adjective_count| adverb_count| noun_count| pronoun_count| verb_count|prediction|
+-------+------------------+--------------------+--------------------+-----------------+------------------+------------------+-----------------+------------------+------------------+-----------------+----------+
 count| 28957| 28957| 28957| 28957| 28957| 28957| 28957| 28957| 28957| 28957| 28957|
 mean|14.997340884760161| 0.14613096956404825| 0.490328654740926|688.8586179507546|30.814310874745313| 62.15191490831233|51.17726283800118| 56.185758193183| 62.15191490831233|130.8608281244604| 2.0|
 stddev| 38.99105294918216| 0.06641134528697305|0.056254164690568466|345.1522578940558|16.784224734667635|33.804255365812665|26.70459670569582|29.233931163476026|33.804255365812665|63.81576094278902| 0.0|
 min| 0|-0.17044513457556937| 0.25526695526695525| 425| 1| 4| 2| 2| 4| 28| 2|
 max| 803| 0.411420361247